In [51]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, SeparableConv2D, BatchNormalization,
                                     Activation, MaxPooling2D, GlobalAveragePooling2D, Add,
                                    Dense, ReLU, Dropout, Concatenate)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import layers, models
import os

from sklearn.utils import class_weight
import numpy as np



In [52]:
# 1. Data setup
img_size = 64
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=10,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    'emotions/train',
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    'emotions/test',
    target_size=(img_size, img_size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

num_classes = len(train_data.class_indices)

Found 1521 images belonging to 6 classes.
Found 748 images belonging to 6 classes.


In [83]:
def build_mobilenetlike_model(input_shape=(64, 64, 1), num_classes=6):
    inputs = Input(shape=input_shape)

    x = SeparableConv2D(32, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    x = SeparableConv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    x = SeparableConv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    x = SeparableConv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(64, activation='relu')(x)  # NEW Dense layer
    x = Dropout(0.05)(x)
    
    x = Dense(32, activation='relu')(x)  # small dense layer
    
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model


In [31]:
def small_res_block(x, filters):
    shortcut = x

    if x.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, padding='same', use_bias=False)(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = SeparableConv2D(filters, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = SeparableConv2D(filters, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x

In [91]:
def build_tiny_resnet(input_shape=(64, 64, 1), num_classes=6):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, 3, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = small_res_block(x, 32)
    x = MaxPooling2D()(x)  # Downsample

    x = small_res_block(x, 64)
    x = MaxPooling2D()(x)

    x = small_res_block(x, 128)
    x = GlobalAveragePooling2D()(x)

    # x = Dense(64, activation='relu')(x)  # NEW Dense layer
    # x = Dropout(0.05)(x)
    
    # x = Dense(32, activation='relu')(x)  # small dense layer

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

In [40]:
def make_shallow_wide_model(input_shape=(64, 64, 1), num_classes=6):
    inputs = layers.Input(shape=input_shape)

    # Shallow Conv Layers with reduced channels
    x = layers.Conv2D(32, (5, 5), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Reducing the complexity further in the last conv layer
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)

    # Reduce dense layer size to keep param count low
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, x)

In [48]:
def fire_module(x, squeeze_channels, expand_channels):
    squeeze = Conv2D(squeeze_channels, (1,1), activation='relu', padding='same')(x)

    expand1x1 = Conv2D(expand_channels, (1,1), activation='relu', padding='same')(squeeze)
    expand3x3 = Conv2D(expand_channels, (3,3), activation='relu', padding='same')(squeeze)

    output = Concatenate(axis=-1)([expand1x1, expand3x3])
    return output

In [55]:
def build_squeezenet(input_shape=(64, 64, 1), num_classes=6):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), strides=2, padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = fire_module(x, squeeze_channels=16, expand_channels=32)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = fire_module(x, squeeze_channels=24, expand_channels=64)
    x = MaxPooling2D(pool_size=(2,2))(x)

    x = fire_module(x, squeeze_channels=32, expand_channels=128)
    x = GlobalAveragePooling2D()(x)

    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs, outputs)

In [107]:
# model = build_mobilenetlike_model()
# model = build_tiny_resnet()
model = make_shallow_wide_model()
# model = build_squeezenet()

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "functional_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_31 (InputLayer)     │ (None, 64, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_88 (Conv2D)              │ (None, 64, 64, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_158         │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_72 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_89 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_159         │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_73 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_90 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_160         │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_28     │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,174 (231.15 KB)

 Trainable params: 58,854 (229.90 KB)

 Non-trainable params: 320 (1.25 KB)

In [57]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))

In [108]:
history = model.fit(
    train_data,
    epochs=50,
    #class_weight=class_weights,
    validation_data=test_data,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    ]
)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - accuracy: 0.2273 - loss: 1.7847 - val_accuracy: 0.1350 - val_loss: 1.8059
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 118ms/step - accuracy: 0.2613 - loss: 1.7361 - val_accuracy: 0.1217 - val_loss: 1.8257
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.2889 - loss: 1.7219 - val_accuracy: 0.1591 - val_loss: 1.8058
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 114ms/step - accuracy: 0.3119 - loss: 1.7003 - val_accuracy: 0.1604 - val_loss: 1.8070
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.3050 - loss: 1.6944 - val_accuracy: 0.1631 - val_loss: 1.8295
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.2859 - loss: 1.6947 - val_accuracy: 0.1832 - val_loss: 1.8045
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 119ms/step - accuracy: 0.3218 - loss: 1.6729 - val_accuracy: 0.1390 - val_loss: 1.8663
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.3305 - loss: 1.6629 - val_accuracy: 0.

In [102]:
train_data.classes
np.bincount(train_data.classes)

array([252, 224, 273, 162, 314, 296])

In [109]:
model.save("shallow_wide.h5")